# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start}\n')
        return result
    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    def wrapper(*args, **kwargs):
        arg_str = ', '.join(repr(arg) for arg in args)
        result = func(*args, **kwargs)
        print(f'Функция вызвана с параметрами:\n({arg_str}), {kwargs}\n')
        return result
    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    def wrapper(*args, **kwargs):
        wrapper.num_calls += 1
        result = func(*args, **kwargs)
        print(f'Функция была вызвана: {wrapper.num_calls} раз\n')
        return result
    wrapper.num_calls = 0
    return wrapper

## Задание 4

In [6]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  def fmemo(*args):
      if args[0] in cache.keys():
          result = cache[args[0]]
      else:
          result = func(*args)
          cache[args[0]] = result
      return result
  fmemo.cache = cache
  return fmemo

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count: 0.42296600099962234

Функция вызвана с параметрами:
('whole'), {}

Функция была вызвана: 1 раз

Cлово whole встречается 176 раз


In [8]:
@benchmark
def fib(n):
    """
    Функция вычисления n-го числа Фибоначчи
    """
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [9]:
# измеряем время выполнения
%%time
fib(20)

Выходные данные были обрезаны до нескольких последних строк (5000).
Время выполнения функции fib: 5.329299983714009e-05

Время выполнения функции fib: 4.049998096888885e-07

Время выполнения функции fib: 4.2800002120202407e-07

Время выполнения функции fib: 4.3400086724432185e-07

Время выполнения функции fib: 5.301500004861737e-05

Время выполнения функции fib: 0.05110759700073686

Время выполнения функции fib: 0.051259539000056975

Время выполнения функции fib: 0.05142020599942043

Время выполнения функции fib: 0.05166790800012677

Время выполнения функции fib: 6.339996616588905e-07

Время выполнения функции fib: 5.840001904289238e-07

Время выполнения функции fib: 4.579997039400041e-07

Время выполнения функции fib: 5.417800002760487e-05

Время выполнения функции fib: 0.00011904900020454079

Время выполнения функции fib: 4.73000000056345e-07

Время выполнения функции fib: 4.929997885483317e-07

Время выполнения функции fib: 5.1839000661857426e-05

Время выполнения функции fib: 4.359

6765

In [10]:
@benchmark
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [11]:
# измеряем время выполнения
%%time
fib(20)

Время выполнения функции fmemo: 2.3400007194140926e-06

Время выполнения функции fmemo: 1.7510001271148212e-06

Время выполнения функции fmemo: 0.00010332199963158928

Время выполнения функции fmemo: 1.2609998520929366e-06

Время выполнения функции fmemo: 9.900004442897625e-07

Время выполнения функции fmemo: 4.973399973096093e-05

Время выполнения функции fmemo: 0.00018836199978977675

Время выполнения функции fmemo: 7.990001904545352e-07

Время выполнения функции fmemo: 8.050001270021312e-07

Время выполнения функции fmemo: 3.768300030060345e-05

Время выполнения функции fmemo: 0.0002607629994599847

Время выполнения функции fmemo: 8.280003385152668e-07

Время выполнения функции fmemo: 8.309998520417139e-07

Время выполнения функции fmemo: 3.454899979260517e-05

Время выполнения функции fmemo: 0.00032998599999700673

Время выполнения функции fmemo: 7.810003808117472e-07

Время выполнения функции fmemo: 7.740000000922009e-07

Время выполнения функции fmemo: 3.35770000674529e-05

Время

6765

Видим многократное ускорение работы функции при кэшировании результатов: 0,01735 с вместо 3,936 с (более, чем в 200 раз).